In [1]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val input = Path("day10.txt").readLines()

typealias Sketch = List<String>
typealias Position = Pair<Int, Int>

# Part 1

In [2]:
fun getResult(sketch: Sketch): Int {
    val animalPosition = sketch.mapIndexed { i, row -> i to row.indexOf('S') }.find { (_, j) -> j >= 0 }!!
    val path = getPath(animalPosition, sketch)
    return path.size / 2
}

fun getPath(startingPosition: Position, sketch: Sketch): List<Position> {
    val (firstConnectingPipe, secondConnectingPipe) = getConnectingPipes(sketch, startingPosition)
    val path = mutableListOf<Position>(startingPosition, firstConnectingPipe)
    while (path.last() != secondConnectingPipe) {
        val connectingPipes = getConnectingPipes(sketch, path.last())
        val nextPipe = connectingPipes.find { it != path.get(path.size - 2) }!!
        path.add(nextPipe)
    }
    return path
}

fun getTile(sketch: Sketch, position: Position) = sketch.getOrNull(position.first)?.getOrNull(position.second)

fun getConnectingPipes(sketch: Sketch, position: Position): List<Position> {
    val (row, column) = position
    val tile = getTile(sketch, position)
    return buildList {
        if (tile == 'S' || tile == '|' || tile == 'L' || tile == 'J') {
            val topTilePosition = row - 1 to column
            val topTile = getTile(sketch, topTilePosition)
            if (topTile == '|' || topTile == '7' || topTile == 'F') {
                add(topTilePosition)
            }
        }


        if (tile == 'S' || tile == '|' || tile == '7' || tile == 'F') {
            val bottomTilePosition = row + 1 to column
            val bottomTile = getTile(sketch, bottomTilePosition)
            if (bottomTile == '|' || bottomTile == 'L' || bottomTile == 'J') {
                add(bottomTilePosition)
            }
        }


        if (tile == 'S' || tile == '-' || tile == '7' || tile == 'J') {
            val leftTilePosition = row to column - 1
            val leftTile = getTile(sketch, leftTilePosition)
            if (leftTile == '-' || leftTile == 'F' || leftTile == 'L') {
                add(leftTilePosition)
            }
        }


        if (tile == 'S' || tile == '-' || tile == 'F' || tile == 'L') {
            val rightTilePosition = row to column + 1
            val rightTile = getTile(sketch, rightTilePosition)
            if (rightTile == '-' || rightTile == '7' || rightTile == 'J') {
                add(rightTilePosition)
            }
        }

    }
}

## Example 1

In [3]:
val example = listOf(
    ".....",
    ".S-7.",
    ".|.|.",
    ".L-J.",
    ".....",
)
getResult(example)


4

## Example 2

In [4]:
val example2 = listOf(
    "..F7.",
    ".FJ|.",
    "SJ.L7",
    "|F--J",
    "LJ..."
)
getResult(example2)

8

## Solution

In [5]:
getResult(input)

7030